# MMVCのリポジトリをGoogle Driveに保存する

ver.2022/12/30

MMVC_Trainerのリポジトリと事前学習済みモデルを、Google Driveに保存します。

​

In [ ]:
#@title ## 0 ノートブックの準備

#@markdown このノートブックのセットアップを行います。セルを実行し、完了したら次に進んでください。

#debug用ディレクトリの作成
!rm -rf /mmvc-debug
!mkdir /mmvc-debug

#現在時刻の取得
import datetime
ｊst_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(jst_delta, 'JST')
now = datetime.datetime.now(JST)
nowt = now.strftime('%Y%m%d%H%M%S')

#python管理
import sys

#出力記録用カスタムマジック %%ccapture
from IPython import get_ipython
from IPython.core import magic_arguments
from IPython.core.magic import register_cell_magic
from IPython.utils.capture import capture_output

@magic_arguments.magic_arguments()
@magic_arguments.argument('output', type=str, default='', nargs='?')

@register_cell_magic
def ccapture(line, cell):
    args = magic_arguments.parse_argstring(ccapture, line)
    with capture_output() as outputs:
        get_ipython().run_cell(cell)
    if args.output:
        get_ipython().user_ns[args.output] = outputs
    
    outputs()

​

In [ ]:
%%ccapture 1-mount-google-drive
print("----------------------------------------------------------------------------------------------------")
print("1 Google Driveをマウント")

#@title ## 1 Google Driveをマウント
#@markdown **このノートブックで、Google Driveを使用するための設定です。**

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown 　「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown 成功すれば、下記メッセージが出ます。

#@markdown ``` 
#@markdown Mounted at /content/drive/
#@markdown ```

from google.colab import drive
drive.mount('/content/drive')

​

In [ ]:
%%ccapture 2-clone-mmvc-trainer
print("----------------------------------------------------------------------------------------------------")
print("2 MMVC_Trainerのクローン")

#@title ## 2 MMVC_Trainerのクローン
#@markdown 以下で指定したディレクトリ上に、MMVC_Trainerというフォルダが作成されます。

#@markdown 実行前に、画面右上にあるアカウントのアイコンをクリックして、アカウントを確認してください。

#@markdown 成功すると、最後に以下のようなメッセージが表示されます。
#@markdown ```
#@markdown Checking out files: 100% (902/902), done.
#@markdown ```

#@markdown ​
#@markdown ### Settings
directory_r = "/content/drive/MyDrive" #@param {type:"string"}
directory = str(directory_r) + "/MMVC_Trainer"

%cd $directory_r
!git clone --depth=1 -b v1.3.3.0 https://github.com/isletennos/MMVC_Trainer/

​

In [ ]:
%%ccapture 3-download-fine-model
print("----------------------------------------------------------------------------------------------------")
print("3 fine_modelのダウンロード")

#@title ## 3 fine_modelのダウンロード
#@markdown 事前学習済みのモデルをダウンロードします。

!curl -L https://dls.srgr0.com/D_180000.pth --output "{directory}/fine_model/D_180000.pth"
!curl -L https://dls.srgr0.com/G_180000.pth --output "{directory}/fine_model/G_180000.pth"

​

In [ ]:
%%ccapture 4-check-files
print("----------------------------------------------------------------------------------------------------")
print("4 確認")

#@title ## 4 確認
#@markdown 以下のようなメッセージが表示されれば、保存が正常に完了しています。
#@markdown ```
#@markdown Directory: MMVC_Trainer
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown
#@markdown Directory: MMVC_Trainer/fine_model
#@markdown D_180000.pth
#@markdown G_180000.pth
#@markdown ```

print("Directory: MMVC_Trainer")
!ls -1 {directory}
print("\n")
print("Directory: MMVC_Trainer/fine_model")
!ls -1 {directory}/fine_model

​

## サポート専用

**以下はお問い合わせの際、指示があった場合のみ使用してください。**

In [ ]:
#@markdown **このセルは無視してください。**

#@markdown このセルは、セルが一括で実行されることを防ぐためのものです。

#@markdown  実行してしまった場合は、左側のアイコンをクリックしてセルを終了してください。

#一括実行の阻止
import time
time.sleep(86400)

​

In [ ]:
#@title ## サポート用ファイルの作成

#@markdown セルを実行すると内部処理が行われ、zipファイルが操作中のPC(またはタブレットなど)にダウンロードされます。

#@markdown ダウンロードされるzipファイルには、以下のファイルや情報が含まれます。

#@markdown * MMVC_Trainerフォルダ内のファイルの一覧
#@markdown * このノートブックで使用されている変数(自動的に設定されるものと、ユーザーが入力するものがあります)のリスト
#@markdown * このセッション内の出力
#@markdown * configsフォルダ内及びlogsフォルダ内のファイル

#@markdown これらには、ユーザーの個人情報が含まれる可能性があります。ダウンロード完了後、ファイルを共有する前に、必ず内容をご確認ください。

#@markdown ファイルのダウンロードが完了したら、ランタイムを切断してください。

#ファイルの準備
variable_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-variable.txt"
tree_dic_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-tree_dic.txt"
export_zip = "/mmvc-debug-" + str(nowt)
export_zipp = "/mmvc-debug-" + str(nowt) + ".zip"

#変数の値を保存
#whos使うと長い文字列が省略されるため、変数毎に取得する
##変数の一覧を取得
vlist = %who_ls
##それぞれの変数で値を取得してファイルに保存
with open(variable_txt, 'w') as f:
  for ev in vlist:
    try:
      #変数名(str)
      print(ev, end=' : ', file=f) 
      #変数の型(変数名がstrとなっているためevalでkeyに直す)
      print(type(eval(ev)), end=' : ', file=f)
      #変数の内容(変数名がstrとなっているためevalでkeyに直す)
      print(eval(ev), file=f)
    except:
      pass

#tree
!apt install tree
##ディレクトリ内以下
import traceback
with open(tree_dic_txt, 'w') as f:
  try:
    !tree {directory} > {tree_dic_txt}
  except Exception as e:
    print("An error occurred!", file=f)
    print(e, file=f)
    print(traceback.print_exc(), file=f)

#logsとconfigsの保存
#directoryが無かったらmkdirしてno-dic.txt置く
!if [ -d {directory}/logs ]; then if [ -z "$(ls {directory}/logs)" ]; then touch {directory}/logs/no-file.txt;else cp {directory}/logs /mmvc-debug/logs; fi;else mkdir /mmvc-debug/logs && touch /mmvc-debug/logs/no-dic.txt; fi
!if [ -d {directory}/configs ]; then if [ -z "$(ls {directory}/configs)" ]; then touch {directory}/configs/no-file.txt;else cp {directory}/configs /mmvc-debug/configs; fi;else mkdir /mmvc-debug/configs && touch /mmvc-debug/configs/no-dic.txt; fi

#直近のtracebackの保存
with open('/mmvc-debug/traceback.txt', 'w') as f:
  print(sys.last_type, sys.last_value, sys.last_traceback, file=f)

#ccaptureの保存
!mkdir /mmvc-debug/ccapture
with open('/mmvc-debug/ccapture/1-mount-google-drive.txt', 'w') as f:
  try:
    print(1-mount-google-drive, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/2-clone-mmvc-trainer.txt', 'w') as f:
  try:
    print(2-clone-mmvc-trainer, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/3-download-fine-model.txt', 'w') as f:
  try:
    print(3-download-fine-model, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/4-check-files.txt', 'w') as f:
  try:
    print(4-check-files, file=f)
  except:
    pass

#zipにまとめる
!apt install zip
!zip {export_zip} -r /mmvc-debug

#colabのfilesモジュールを使ってダウンロード
from google.colab import files
files.download(export_zipp)